# ASSIGNMENT 5

Chapter 6, Question 9
6. a) Split into training and test sit
   
   b) Run OLS on training set and report the test error
   
   c)Fit ridge regression with $\lambda$ chosen by CV. Report the test error
   
   d)Fit a lasso model on the training set with $\lambda$ chosen by CV. Report the test error.
   
   e)Fit PCR on the training set, with M chosen by cross-validation. Report test and M
   
   f)Fit PLS model , on the training set, with M chosen by cross-validation. Report test error and M
   
   g)Comment on the result obtained, how accuratley can we predict MedianValue.
   
  # A) Split into training and test

In [ ]:
from sklearn import datasets
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error


boston = datasets.load_boston()
X = pd.DataFrame( boston.data)
y = pd.DataFrame(boston.target)
X.columns = boston.feature_names
X["MedVal"] =y

training_set,test_set = train_test_split(X, test_size=0.2,random_state=42)
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MedVal
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


# B) Run OLS on the training and report test error

In [123]:
params = list( X.columns[0:13])
target = "MedVal"
reg = sm.OLS(training_set[target],training_set[params]).fit()
y_hat = reg.predict(test_set[params])
error_ols = mean_squared_error(test_set[target], y_hat)
print("Test MSE: " + str(error_ols))

Test MSE: 29.04404335035871


# C)Fit ridge regression with  λ  chosen by CV. Report the test error

In [124]:
from sklearn.linear_model import RidgeCV

alphas = np.linspace(.00001, 2, 500)
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(training_set[params],training_set[target])
ridge_predict = ridgecv.predict(test_set[params])
error_ridge = mean_squared_error(test_set[target], ridge_predict)
print("Test MSE: " + str(error_ridge))
print("Lambda: " + str(ridgecv.alpha_))

Test MSE: 24.42849154209667
Lambda: 0.01604198396793587


# D)Fit a lasso model on the training set with $\lambda$ chosen by CV. Report the test error and lambda.

In [125]:
from sklearn.linear_model import LassoCV
lassocv = LassoCV(alphas = alphas, cv=20,max_iter=10000,normalize=True).fit(training_set[params], training_set[target])
lasso_predict = lassocv.predict(test_set[params])
error_lasso  = mean_squared_error(test_set[target], lasso_predict)
print("Test MSE: " + str(error_lasso))
print("Lambda: " + str(lassocv.alpha_))

Test MSE: 24.29179936268873
Lambda: 1e-05


# E) Fit PCR with CV, Report test error and M

Note: StackExchange was invaluable here

In python, we don't have cross validation and PCR in the nice package. So instead it has to be done manually. The standard way to do this is to chain together operations in a "pipeline".


In [126]:
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

#Scaling the data
X_scaled = scale(training_set[params])
y_scaled = scale(training_set[target])

X_test_scaled = scale(test_set[params])
y_test_scaled = scale(test_set[target])


So I finally figured it out using a hack. Creating a selectColumns function with a n_columns param is actually really hard. Theres all kinds of weird inheritence that goes on, strictly speaking it is called duck typing and I don't understand it. Instead I run PCA EVERY SINGLE TIME during cross validation, but with a different n_components value.

In [137]:
from sklearn.metrics import mean_squared_error
 
linear = LinearRegression()
pca = PCA()

pipe = Pipeline(steps=[('pca', pca), ('linear', linear)])
#Setting up parameter grid, a dictionary, Note: Naming of string is imporant 
param_grid = {
    'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13],
}



search = GridSearchCV(pipe, param_grid, iid=False, cv=20,return_train_score=False)
search.fit(X_scaled, training_set[target])
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

error_pcr =mean_squared_error(search.best_estimator_.predict(X_test_scaled), test_set[target])

print("Test MSE: " + str(error_pcr))


Best parameter (CV score=0.683):
{'pca__n_components': 13}
Test MSE: 27.406836284337277


# F) Fit PLS with CV, Report test error and M

An issue I ran into here is that the string 'pls__n_components' actually provides information. Misspecifying it results in an error.


In [136]:
from sklearn.cross_decomposition import PLSRegression
pls = PLSRegression()

param_grid = {
    'n_components': [2,3,4,5,6,7,8,9,10,11,12],
}


search_pls = GridSearchCV(pls , param_grid, iid=False, cv=20,return_train_score=False)
search_pls.fit(X_scaled, training_set[target])

print("Best parameter (CV score=%0.3f):" % search_pls.best_score_)
print(search_pls.best_params_)

error_pls =mean_squared_error(search_pls.best_estimator_.predict(X_test_scaled), test_set[target])
print("Test MSE: " + str(error_pls))

Best parameter (CV score=0.683):
{'n_components': 8}
Test MSE: 27.58066477982635


# Comment On Findings

Lasso and Ridge Regression both perform really well in terms of mean-squared-error. PCR,PLS perform better than OLS and have near identical MSEs; however, they differ signifigantly in terms of optimal number of paramters. I'm not quite sure what that means.  

How well can we predict median value?
On average we are off by 24,000 which could be signifigant depending on how valuable the neighbourhood is. If med_val is 100,000, this model could be off by 25%, which strikes me as actually very poor performance. 